# New Hampshire

Runs the entire RBA pipeline on New Hampshire geodata and districts.

The whole notebook should take a couple minutes to run. (requires ffmpeg)

In [ ]:
# CONSTANTS

# Input paths
GEODATA_FILE = "../rba/data/2010/new_hampshire_geodata_merged.json"
DISTRICTS_FILE = "../rba/data/2010/new_hampshire_districts.json"
VRA_FILE = "../rba/data/2010/vra_new_hampshire.json"

# Output paths
COMMUNITIES_FILE = "../rba/data/2010/new_hampshire_communities.json"
ENSEMBLE_DIR = "../NH_demo_ensemble_out"
COMMUNITY_VIS_DIR = "../NH_demo_supercommunities_out"
OPTIMIZATION_DIR = "../NH_demo_optimization_out"

# Algorithm parameters
NUM_THRESHOLDS = 1000
ENSEMBLE_SIZE = 100
NUM_OPTIMIZATION_STEPS = 100
NUM_DISTRICTS = 2

# Visualization parameters
NUM_SUPERCOMMUNITY_FRAMES = 50

In [ ]:
# Generate supercommunity map

import warnings
warnings.filterwarnings("ignore")

from rba.community_generation import create_communities

create_communities(GEODATA_FILE, NUM_THRESHOLDS, COMMUNITIES_FILE, verbose=True)

In [ ]:
# Visualize supercommunities (this takes most of the time to run)

import json

from gerrychain import Graph
import networkx as nx

from rba.visualization import visualize_community_generation

with open(GEODATA_FILE, "r") as f:
    data = json.load(f)
nx_graph = nx.readwrite.json_graph.adjacency_graph(data)
graph = Graph.from_networkx(nx_graph)
del nx_graph

visualize_community_generation(COMMUNITIES_FILE, COMMUNITY_VIS_DIR, graph, NUM_SUPERCOMMUNITY_FRAMES)

In [ ]:
!ffmpeg -framerate 2 -pattern_type glob -i "../NH_demo_supercommunities_out/*.png" -c:v libx264 ../NH_demo_supercommunities_out/out.mp4

In [ ]:
# Generate an ensemble

from rba.ensemble import ensemble_analysis


ensemble_analysis(GEODATA_FILE, COMMUNITIES_FILE, VRA_FILE, ENSEMBLE_SIZE, NUM_DISTRICTS,
                  DISTRICTS_FILE, DISTRICTS_FILE, ENSEMBLE_DIR, verbose=True)

In [ ]:
# Run optimization

from rba.optimization import optimize

optimize(GEODATA_FILE, COMMUNITIES_FILE, VRA_FILE, NUM_OPTIMIZATION_STEPS, NUM_DISTRICTS,
         DISTRICTS_FILE, OPTIMIZATION_DIR, verbose=True)

In [ ]:
!python3 ../rba/scripts/optimization_plots.py new_hampshire 100 2 ../NH_demo_ensemble_out ../NH_demo_optimization_out